In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np

from utils import loss
from dataloader import IntentExample
from dataloader import load_intent_examples
from dataloader import sample


top param: [[73.33151077 12.35407765]
 [ 2.63710441 30.25258676]]
bottom_param: 118.57527958882264
-0.3333333333333333


In [2]:
!pwd

/home/sitindustry/Documents/Reading/implement_papers


In [7]:
# Few-Shot Intent Detection via Contrastive Pre-Training and Fine-Tuning
# https://github.com/jianguoz/Few-Shot-Intent-Detection

## Todo
 1. load data and combined them from every sources
 2. preprocess -> embedded two sentence into single encoder -> h
 3. use h to calculate contrasive learning loss 
* dont forget to use nn.CosineSimilarity instead of from scartch

## Parameters
- N - number of sentences in the batch 
- t - temperature paramter -> control penalty for neg samples
- u - sentecnce or utterances 
- hi - representation of ui
- hi - BERT(ui) -> we use bert-base-uncased
- i-th -> order of sentence
- hi_bar -> the representation of ui_bar 
- ui_bar -> the same sentence as ui but (10%) tokens are random masked  (Devlin et al.,2019)
- M - the number of maksed tokens in each batch 

## Preprocessesing data
  Todo 
  1. dont forget to remove utterances that less than five tokens
  2. we will all exclude  CLINC-Single-Domain-OOS

## Trainning process
 1. during batch trainning dont forget to randomly maksed, a sentence has 
    different maksed postion across different trannig epochs
 2. (ui ,ui_bar) -> single encoder during batch tranning  (Gao et al., 2021)
 3. add mask language modelling loss  (Devlinet al., 2019; Wu et al., 2020a)
 4. P(xm) -> predicted probabilty of maksed token xm over total vocabulary

## Experiment Setting
- contrasive Pretraining 
 1. Pre-train the combined intent datasets -> combine every dataset (guess)
 2. 15 epochs
 3. batch size = 64
 4. t = 0.1 , lamda = 1.0

- Fine tunning 
 1. 5-shot -> five trainning examples per intents
 2. 10-shot -> tens trainning examples per intents
 3. batch size = 16
 4. t =  {0.1, 0.3, 0.5}
 5. λ ∈ {0.01, 0.03, 0.05}
 6. 30 epochs
 7. apply label smoothing to the intent classifcation loss (Zhang et al. (2020a)

## Load dataset 

In [20]:
!ls ../../dataset/Few-Shot-Intent-Detection/Datasets/CLINC150/oos/train/

label  seq.in


In [53]:
N = 5  # samples per class
T = 10 # number of class 

In [13]:
train_file_path = '../../dataset/Few-Shot-Intent-Detection/Datasets/CLINC150/train/'
#print(data.read())

In [54]:
train_examples = load_intent_examples(train_file_path)
sampled_tasks = [sample(N, train_examples) for i in range(T)]

In [67]:
# we have 15000 instance both label and text 

In [68]:
train_examples[0].original_text

'what expression would i use to say i love you if i were an italian'

In [ ]:
train_examples[0].text

In [64]:
c =0

In [65]:
for e in train_examples:
    c = c+1
print(c)

15000


In [55]:
num_train = len(train_examples)
print(num_train)

15000


In [15]:
type(sampled_tasks)

list

In [28]:
print(type(sampled_tasks)) , print(type(sampled_tasks[0][0]))

<class 'list'>
<class 'dict'>


(None, None)

In [56]:
len(sampled_tasks)

10

In [33]:
len(sampled_tasks[0][0]["examples"]) 

5

In [ ]:
#guess 150 

In [44]:
sampled_tasks[9][149]

{'task': 'card_declined',
 'examples': ['please help me figure out why my card was declined yesterday',
  'so why was my card declined yesterday',
  'where can i find out why my card was recently declined at amazoncom',
  'for what reason did my card get declined',
  "i need to know why my card was just declined at walgreen's"]}

In [34]:
len(sampled_tasks[0]), len(sampled_tasks[0][0])

(150, 2)

In [ ]:
#(list(list(dict(

In [22]:
sampled_tasks

[[{'task': 'translate',
   'examples': ['how do they say "what\'s up" in chinese',
    'do you know how to say where is the restroom in french',
    'how do you say hi in french',
    "how do you say i'm hungry in french",
    'how do you say cat in spanish']},
  {'task': 'transfer',
   'examples': ['transfer $500 from my money market savings account to my checking account',
    'move my money please',
    'i would like to transfer from one account to my second one',
    'please transfer my funds',
    'i need to move money from one account to another']},
  {'task': 'timer',
   'examples': ['start the timer now and stop at thirty minutes',
    'please set a timer for me',
    'set a timer to go off in 5 minutes',
    'set a seven minute timer',
    'set a timer for fifteen seconds']},
  {'task': 'definition',
   'examples': ['what is stupedous meant',
    'tell me what the word arboreal means',
    'i want to know the definition of discretionary',
    'do you know what calumny means  p

In [ ]:
# https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial

## Load Pretrain model from vocabuary

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Tokenize input 
text = "who was Elon musk ? Elon musk was an entrepreneur and business magnate"
tokenized_text = tokenizer.tokenize(text)

In [7]:
# Mask a token that we will try to predict with MLM
masked_index = 7
tokenized_text[masked_index] = '[MASK]'
#assert tokenized_text == [who was Elon musk ? Elon musk was an entrepreneur 'and','business','magnate']

In [8]:
print(len(tokenized_text))
tokenized_text

17


['who',
 'was',
 'el',
 '##on',
 'mu',
 '##sk',
 '?',
 '[MASK]',
 '##on',
 'mu',
 '##sk',
 'was',
 'an',
 'entrepreneur',
 'and',
 'business',
 'magnate']

In [9]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd 
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

## Load weight Pretrain model

In [10]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
print("eval done")

eval done


In [11]:
# Predict hidden states features for each layer
encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [12]:
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

In [13]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
#assert predicted_token == 'musk'

In [14]:
predicted_token

['el']

## Config

In [44]:
N = 10 #None
temperature = 0.5  #None

In [74]:
a = np.random.rand(3,2)
b = np.random.rand(3,2)

In [75]:
a.shape , b.shape

((3, 2), (3, 2))

In [76]:
self_supervised_cl(a,b)

[[1.92643996 1.70838819]
 [3.35620428 2.74840212]]
9.7394345458278


-0.1

In [ ]:
predicted_token